# OpenAI Agents SDK - 오케스트레이션 패턴

이 튜토리얼은 OpenAI Agents SDK에서 멀티 에이전트 워크플로우를 오케스트레이션하는 다양한 패턴을 다룬다. 코드 기반 결정론적 워크플로우, 하이브리드 오케스트레이션, 공유 컨텍스트 패턴 등의 고급 기법을 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv pydantic

---

## 1. 코드 기반 오케스트레이션

이 챕터에서는 결정론적 멀티 에이전트 워크플로우를 구축하는 방법을 다룬다.

### 학습 내용

- 구조화된 출력을 사용한 워크플로우 라우팅 제어
- 명시적 제어 흐름을 가진 결정론적 에이전트 파이프라인 구축
- 각 워크플로우 단계에서의 에러 처리 및 검증
- 에이전트 간 구조화된 데이터 전달을 통한 순차 처리

### 환경 설정

In [16]:
import os
import asyncio
import json
from datetime import datetime
from dotenv import load_dotenv
from pydantic import BaseModel, ConfigDict
from typing import Dict, Any, Optional, List
from agents import Agent, Runner

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### 처리 결과 모델 정의

워크플로우에서 에이전트 간 데이터를 전달하기 위한 구조화된 출력 모델을 정의한다.

In [2]:
class ProcessingResult(BaseModel):
    """
    워크플로우 처리 결과를 담는 모델이다.
    각 에이전트는 이 형식으로 결과를 반환한다.
    """
    model_config = ConfigDict(strict=True)
    
    success: bool                          # 처리 성공 여부
    data: Dict[str, Any]                   # 처리된 데이터
    next_agent: str                        # 다음 에이전트 이름
    error_message: Optional[str] = None   # 에러 메시지 (실패 시)

print("ProcessingResult 모델이 정의되었다.")

ProcessingResult 모델이 정의되었다.


### 워크플로우 에이전트 생성

데이터 수집, 분석, 보고서 생성을 담당하는 세 개의 에이전트를 생성한다.

In [3]:
# 데이터 수집 에이전트
data_collector_agent = Agent(
    name="Data Collector",
    instructions="""
    당신은 입력 데이터를 수집하고 검증하는 에이전트입니다.
    
    반환 형식:
    - success: 데이터가 유효하면 true
    - data: 처리/정제된 데이터
    - next_agent: 데이터가 완전하면 "analyzer", 유효하지 않으면 "error"
    - error_message: 검증 실패 시 에러 메시지
    
    필수 필드: customer_id, purchase_amount, product, date
    """,
    output_type=ProcessingResult    
)

# 데이터 분석 에이전트
analyzer_agent = Agent(
    name="Data Analyzer", 
    instructions="""
    당신은 수집된 데이터를 분석하고 인사이트를 생성하는 에이전트입니다.
    
    반환 형식:
    - success: 분석이 완료되면 true
    - data: 분석 결과와 인사이트
    - next_agent: 분석 성공 시 "reporter", 실패 시 "error"
    - error_message: 분석 실패 시 에러 메시지
    """,
    output_type=ProcessingResult
)

# 보고서 생성 에이전트
reporter_agent = Agent(
    name="Report Generator",
    instructions="""
    당신은 분석된 데이터로부터 최종 보고서를 생성하는 에이전트입니다.
    
    반환 형식:
    - success: 보고서가 생성되면 true
    - data: 최종 보고서 내용
    - next_agent: 완료 시 "complete"
    - error_message: 보고서 생성 실패 시 에러 메시지
    """,
    output_type=ProcessingResult
)

print("워크플로우 에이전트가 생성되었다:")
print(f"  - {data_collector_agent.name}")
print(f"  - {analyzer_agent.name}")
print(f"  - {reporter_agent.name}")

워크플로우 에이전트가 생성되었다:
  - Data Collector
  - Data Analyzer
  - Report Generator


### 오케스트레이션 함수

여러 에이전트를 통해 결정론적 워크플로우를 오케스트레이션하는 함수를 구현한다.

In [4]:
async def orchestrate_workflow(input_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    여러 에이전트를 통해 결정론적 워크플로우를 오케스트레이션한다.
    
    Args:
        input_data: 처리할 입력 데이터
    
    Returns:
        최종 처리된 컨텍스트
    """
    
    current_agent = data_collector_agent
    context = input_data.copy()
    step = 1
    
    print("🔄 코드 기반 오케스트레이션 시작")
    print("=" * 50)
    
    while current_agent:
        agent_name = current_agent.name
        print(f"\n📍 단계 {step}: {agent_name}")
        print(f"입력 컨텍스트: {json.dumps(context, indent=2, ensure_ascii=False)}")
        
        # 구조화된 출력으로 현재 에이전트 실행
        result = await Runner.run(
            current_agent,
            f"이 데이터를 처리하세요: {json.dumps(context, ensure_ascii=False)}"
        )
        
        processing_result = result.final_output
        print(f"에이전트 결과: success={processing_result.success}, next_agent={processing_result.next_agent}")
        
        # 에러 확인
        if not processing_result.success:
            print(f"❌ {agent_name}에서 워크플로우 실패: {processing_result.error_message}")
            raise Exception(f"처리 실패: {processing_result.error_message}")
        
        # 새 데이터로 컨텍스트 업데이트
        context.update(processing_result.data)
        
        # 구조화된 출력에 기반하여 다음 에이전트 결정
        next_agent_name = processing_result.next_agent
        
        if next_agent_name == "analyzer":
            current_agent = analyzer_agent
        elif next_agent_name == "reporter":
            current_agent = reporter_agent
        elif next_agent_name == "complete":
            current_agent = None  # 워크플로우 종료
        elif next_agent_name == "error":
            raise Exception(f"에이전트 {agent_name}이 에러 표시: {processing_result.error_message}")
        else:
            raise Exception(f"알 수 없는 다음 에이전트: {next_agent_name}")
        
        step += 1
        print("-" * 30)
    
    print("\n✅ 워크플로우 성공적으로 완료")
    return context

print("orchestrate_workflow 함수가 정의되었다.")

orchestrate_workflow 함수가 정의되었다.


### 코드 기반 오케스트레이션 테스트

In [5]:
async def demo_orchestration():
    """
    코드 기반 오케스트레이션을 시연한다.
    """
    
    test_scenarios = [
        {
            "name": "유효한 고객 데이터",
            "input": {
                "customer_id": "12345",
                "purchase_amount": 299.99,
                "product": "프리미엄 소프트웨어 라이선스",
                "date": "2024-01-15"
            }
        },
        {
            "name": "유효하지 않은 데이터 (필수 필드 누락)",
            "input": {
                "customer_id": "67890",
                "product": "기본 라이선스"
                # purchase_amount와 date 누락
            }
        }
    ]
    
    for scenario in test_scenarios:
        print(f"\n{'='*60}")
        print(f"테스트 시나리오: {scenario['name']}")
        print(f"입력: {scenario['input']}")
        print("=" * 60)
        
        try:
            final_result = await orchestrate_workflow(scenario["input"])
            print(f"\n🎉 최종 결과:")
            print(json.dumps(final_result, indent=2, ensure_ascii=False))
            
        except Exception as e:
            print(f"\n💥 워크플로우 에러: {str(e)}")

# 데모 실행
await demo_orchestration()


테스트 시나리오: 유효한 고객 데이터
입력: {'customer_id': '12345', 'purchase_amount': 299.99, 'product': '프리미엄 소프트웨어 라이선스', 'date': '2024-01-15'}
🔄 코드 기반 오케스트레이션 시작

📍 단계 1: Data Collector
입력 컨텍스트: {
  "customer_id": "12345",
  "purchase_amount": 299.99,
  "product": "프리미엄 소프트웨어 라이선스",
  "date": "2024-01-15"
}

💥 워크플로우 에러: Strict JSON schema is enabled, but the output type is not valid. Either make the output type strict, or wrap your type with AgentOutputSchema(YourType, strict_json_schema=False)

테스트 시나리오: 유효하지 않은 데이터 (필수 필드 누락)
입력: {'customer_id': '67890', 'product': '기본 라이선스'}
🔄 코드 기반 오케스트레이션 시작

📍 단계 1: Data Collector
입력 컨텍스트: {
  "customer_id": "67890",
  "product": "기본 라이선스"
}

💥 워크플로우 에러: Strict JSON schema is enabled, but the output type is not valid. Either make the output type strict, or wrap your type with AgentOutputSchema(YourType, strict_json_schema=False)


### 단순 순차 처리 (대안)

복잡한 오케스트레이션 없이 단순한 순차 처리 방식도 가능하다.

In [ ]:
async def simple_sequential_workflow(data: Dict[str, Any]) -> str:
    """
    복잡한 오케스트레이션 없는 단순 순차 처리이다.
    
    Args:
        data: 처리할 데이터
    
    Returns:
        처리 결과 문자열
    """
    
    print("\n🔗 단순 순차 워크플로우 시작")
    
    # 단계 1: 수집
    print("  1단계: 데이터 수집...")
    step1 = await Runner.run(
        data_collector_agent, 
        f"검증: {json.dumps(data, ensure_ascii=False)}"
    )
    if not step1.final_output.success:
        return f"수집 실패: {step1.final_output.error_message}"
    print(f"     ✓ 수집 완료")
    
    # 단계 2: 분석
    print("  2단계: 데이터 분석...")
    step2 = await Runner.run(
        analyzer_agent, 
        f"분석: {json.dumps(step1.final_output.data, ensure_ascii=False)}"
    )
    if not step2.final_output.success:
        return f"분석 실패: {step2.final_output.error_message}"
    print(f"     ✓ 분석 완료")
    
    # 단계 3: 보고
    print("  3단계: 보고서 생성...")
    step3 = await Runner.run(
        reporter_agent, 
        f"보고: {json.dumps(step2.final_output.data, ensure_ascii=False)}"
    )
    if not step3.final_output.success:
        return f"보고 실패: {step3.final_output.error_message}"
    print(f"     ✓ 보고서 생성 완료")
    
    return f"순차 워크플로우 완료: {step3.final_output.data}"

# 테스트
test_data = {
    "customer_id": "TEST-001",
    "purchase_amount": 150.00,
    "product": "스탠다드 플랜",
    "date": "2024-02-01"
}

result = await simple_sequential_workflow(test_data)
print(f"\n결과: {result}")

---

## 2. 하이브리드 오케스트레이션

이 챕터에서는 AI 지능과 비즈니스 규칙을 결합하는 방법을 다룬다.

### 학습 내용

- LLM 결정과 프로그래밍 오버라이드를 결합한 스마트 라우팅 시스템 구축
- 지능형 요청 분류를 위한 라우터 에이전트 생성
- 중요 우선순위와 에스컬레이션을 위한 비즈니스 규칙 오버라이드 구현
- AI 유연성과 결정론적 비즈니스 로직의 균형

### 컨텍스트 및 라우팅 모델

In [11]:
class TaskContext(BaseModel):
    """작업 컨텍스트를 담는 모델이다."""
    model_config = ConfigDict(strict=True)
    
    user_id: str                                      # 사용자 ID
    priority: str                                     # 우선순위 (normal, high, critical)
    category: str                                     # 카테고리
    escalation_level: int = 0                         # 에스컬레이션 레벨
    conversation_history: List[Dict[str, str]] = []   # 대화 기록

class RoutingDecision(BaseModel):
    """라우팅 결정을 담는 모델이다."""
    model_config = ConfigDict(strict=True)
    
    agent_category: str         # 에이전트 카테고리
    confidence: float           # 신뢰도 (0.0 ~ 1.0)
    reasoning: str              # 라우팅 이유
    requires_escalation: bool = False  # 에스컬레이션 필요 여부

print("TaskContext와 RoutingDecision 모델이 정의되었다.")

TaskContext와 RoutingDecision 모델이 정의되었다.


### 전문화된 에이전트 생성

In [12]:
# 일반 지원 에이전트
support_agent = Agent(
    name="Support Agent",
    instructions="""
    일반적인 고객 지원 문의를 도움이 되는 응답으로 처리하세요.
    친절하고 전문적으로 응대하세요.
    """
)

# 기술 지원 에이전트
technical_agent = Agent(
    name="Technical Agent", 
    instructions="""
    기술적 문제, 버그, 문제 해결 요청을 처리하세요.
    단계별 해결 방법을 제공하세요.
    """
)

# 매니저 에이전트
manager_agent = Agent(
    name="Manager Agent",
    instructions="""
    관리자 주의가 필요한 중요한 문제와 복잡한 고객 문제를 처리하세요.
    고객 만족을 최우선으로 하세요.
    """
)

# 에스컬레이션 에이전트
escalation_agent = Agent(
    name="Escalation Agent",
    instructions="""
    여러 번 해결 시도가 실패한 에스컬레이션된 케이스를 처리하세요.
    특별한 관심과 해결책을 제공하세요.
    """
)

print("전문화된 에이전트가 생성되었다:")
print(f"  - {support_agent.name}")
print(f"  - {technical_agent.name}")
print(f"  - {manager_agent.name}")
print(f"  - {escalation_agent.name}")

전문화된 에이전트가 생성되었다:
  - Support Agent
  - Technical Agent
  - Manager Agent
  - Escalation Agent


### 스마트 라우터 에이전트

In [13]:
# LLM 기반 결정을 내리는 스마트 라우터
router_agent = Agent(
    name="Smart Router",
    instructions="""
    고객 요청을 분석하고 적절한 에이전트 카테고리로 라우팅하세요.
    
    카테고리:
    - "support": 일반 질문, 계정 문제, 기본 도움
    - "technical": 버그, 오류, 설치 문제
    - "manager": 불만, 환불, 복잡한 문제
    - "escalation": 이전에 실패한 케이스로 특별한 주의 필요
    
    사용자의 톤, 문제의 복잡성, 긴급성을 고려하세요.
    """,
    output_type=RoutingDecision
)

print(f"스마트 라우터 '{router_agent.name}'가 생성되었다.")

스마트 라우터 'Smart Router'가 생성되었다.


### 하이브리드 라우팅 시스템

In [14]:
def get_agent_by_category(category: str) -> Agent:
    """카테고리를 실제 에이전트로 매핑한다."""
    agent_map = {
        "support": support_agent,
        "technical": technical_agent, 
        "manager": manager_agent,
        "escalation": escalation_agent
    }
    return agent_map.get(category, support_agent)

async def smart_routing_system(user_input: str, context: TaskContext) -> str:
    """
    하이브리드 오케스트레이션: LLM 지능 + 비즈니스 규칙
    
    Args:
        user_input: 사용자 입력
        context: 작업 컨텍스트
    
    Returns:
        최종 응답
    """
    
    print(f"🧠 하이브리드 오케스트레이션")
    print(f"사용자: {context.user_id} | 우선순위: {context.priority} | 에스컬레이션: {context.escalation_level}")
    print(f"입력: {user_input}")
    
    # 단계 1: LLM을 사용하여 초기 분류 및 라우팅
    print("\n1️⃣ LLM 기반 라우팅 결정...")
    router_result = await Runner.run(
        router_agent,
        f"이 요청을 라우팅하세요: {user_input}"
    )
    
    llm_decision = router_result.final_output
    print(f"LLM 제안: {llm_decision.agent_category} (신뢰도: {llm_decision.confidence:.2f})")
    print(f"이유: {llm_decision.reasoning}")
    
    # 단계 2: 비즈니스 규칙 적용 (프로그래밍 오버라이드)
    print("\n2️⃣ 비즈니스 규칙 적용...")
    
    if context.priority == "critical" and llm_decision.agent_category != "manager":
        # 중요 문제에 대한 LLM 결정 오버라이드
        target_agent = manager_agent
        print("🚨 오버라이드: 중요 우선순위 → Manager Agent")
        
    elif context.escalation_level > 2:
        # 여러 번 시도 후 자동 에스컬레이션
        target_agent = escalation_agent
        print("⬆️ 오버라이드: 높은 에스컬레이션 레벨 → Escalation Agent")
        
    elif llm_decision.requires_escalation:
        # LLM이 에스컬레이션 필요 감지
        target_agent = escalation_agent
        print("🤖 LLM 에스컬레이션: 에스컬레이션 필요 감지 → Escalation Agent")
        
    else:
        # LLM의 라우팅 결정 신뢰
        target_agent = get_agent_by_category(llm_decision.agent_category)
        print(f"✅ LLM 결정 따름: {llm_decision.agent_category} agent 사용")
    
    # 단계 3: 선택된 에이전트로 실행
    print(f"\n3️⃣ {target_agent.name}로 실행 중...")
    result = await Runner.run(target_agent, user_input)
    
    return result.final_output

print("smart_routing_system 함수가 정의되었다.")

smart_routing_system 함수가 정의되었다.


### 하이브리드 오케스트레이션 테스트

In [15]:
async def demo_hybrid_orchestration():
    """
    하이브리드 오케스트레이션 시나리오를 시연한다.
    """
    
    test_scenarios = [
        {
            "input": "계정에 로그인할 수 없어요",
            "context": TaskContext(
                user_id="user123",
                priority="normal", 
                category="support",
                escalation_level=0
            ),
            "description": "일반 지원 요청 - LLM 라우팅 따름"
        },
        {
            "input": "앱이 크래시되고 모든 데이터를 잃었어요!",
            "context": TaskContext(
                user_id="user456",
                priority="critical",
                category="technical", 
                escalation_level=0
            ),
            "description": "중요 문제 - 비즈니스 규칙이 매니저로 오버라이드"
        },
        {
            "input": "같은 문제로 세 번째 문의입니다",
            "context": TaskContext(
                user_id="user789",
                priority="normal",
                category="support",
                escalation_level=3
            ),
            "description": "높은 에스컬레이션 레벨 - 에스컬레이션 에이전트로 라우팅"
        },
        {
            "input": "구독을 취소하고 전액 환불받고 싶어요",
            "context": TaskContext(
                user_id="user101",
                priority="normal",
                category="billing",
                escalation_level=0
            ),
            "description": "복잡한 요청 - LLM이 매니저 필요 감지"
        }
    ]
    
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n{'='*80}")
        print(f"시나리오 {i}: {scenario['description']}")
        print("=" * 80)
        
        response = await smart_routing_system(scenario["input"], scenario["context"])
        
        print(f"\n🎯 최종 응답: {response[:200]}..." if len(response) > 200 else f"\n🎯 최종 응답: {response}")
        print("-" * 80)

# 데모 실행
await demo_hybrid_orchestration()


시나리오 1: 일반 지원 요청 - LLM 라우팅 따름
🧠 하이브리드 오케스트레이션
사용자: user123 | 우선순위: normal | 에스컬레이션: 0
입력: 계정에 로그인할 수 없어요

1️⃣ LLM 기반 라우팅 결정...
LLM 제안: support (신뢰도: 0.95)
이유: 사용자는 계정에 로그인할 수 없는 문제를 제기했습니다. 이는 일반적으로 계정 관련 기본 지원 범주에 해당하며, 복잡하거나 긴급, 기술적 문제, 불만 등으로 보이지 않습니다.

2️⃣ 비즈니스 규칙 적용...
✅ LLM 결정 따름: support agent 사용

3️⃣ Support Agent로 실행 중...

🎯 최종 응답: 불편을 드려 죄송합니다. 계정에 로그인할 수 없는 상황이라면 아래 내용을 확인해 주세요:

1. **아이디(이메일)와 비밀번호가 정확한지 다시 한 번 입력해 주세요.**  
   - 대·소문자 구분에 유의해 주세요.

2. **비밀번호를 잊으셨다면, [비밀번호 찾기/재설정] 링크를 이용해 비밀번호를 다시 설정해 주세요.**

3. **계정이 잠겼거나 이상 ...
--------------------------------------------------------------------------------

시나리오 2: 중요 문제 - 비즈니스 규칙이 매니저로 오버라이드
🧠 하이브리드 오케스트레이션
사용자: user456 | 우선순위: critical | 에스컬레이션: 0
입력: 앱이 크래시되고 모든 데이터를 잃었어요!

1️⃣ LLM 기반 라우팅 결정...
LLM 제안: technical (신뢰도: 0.95)
이유: 고객이 앱의 크래시와 데이터 손실을 보고하고 있습니다. 이는 버그 또는 시스템 오류와 관련된 기술적인 문제로 보이며, 신속한 기술 지원이 요구됩니다. 요청의 긴박성도 높습니다.

2️⃣ 비즈니스 규칙 적용...
🚨 오버라이드: 중요 우선순위 → Manager Agent

3️⃣ Manager Agent로 실행 중...

🎯 최종 응답: 앱 사용

---

## 3. 공유 컨텍스트 패턴

이 챕터에서는 여러 에이전트 간에 공유 상태를 유지하는 방법을 다룬다.

### 학습 내용

- Pydantic 모델을 사용한 공유 비즈니스 객체(지원 티켓) 표현
- 상태 변경을 보존하면서 에이전트 간 컨텍스트 전달
- 에이전트들이 동일한 데이터 객체에서 협업하는 워크플로우 구축
- 에이전트 핸드오프 간 에스컬레이션 기록 및 해결 노트 추적

### Support Ticket 모델

In [17]:
class SupportTicket(BaseModel):
    """
    공유 비즈니스 객체로서의 지원 티켓 모델이다.
    여러 에이전트가 이 객체를 수정하며 협업한다.
    """
    model_config = ConfigDict(strict=True)
    
    ticket_id: str                              # 티켓 ID
    customer_id: str                            # 고객 ID
    issue_type: str                             # 문제 유형
    priority: str                               # 우선순위
    status: str                                 # 상태
    description: str                            # 문제 설명
    resolution_notes: List[str] = []            # 해결 노트
    escalation_history: List[str] = []          # 에스컬레이션 기록
    escalation_level: int = 0                   # 에스컬레이션 레벨
    created_at: datetime                        # 생성 시간
    updated_at: datetime                        # 업데이트 시간
    
    def add_note(self, agent_name: str, note: str):
        """에이전트로부터 해결 노트를 추가한다."""
        self.resolution_notes.append(f"[{agent_name}] {note}")
        self.updated_at = datetime.now()
    
    def escalate_to(self, agent_name: str, reason: str):
        """티켓을 다른 에이전트로 에스컬레이션한다."""
        self.escalation_history.append(f"{agent_name}로 에스컬레이션: {reason}")
        self.escalation_level += 1
        self.updated_at = datetime.now()

print("SupportTicket 모델이 정의되었다.")

SupportTicket 모델이 정의되었다.


### 티켓 처리 에이전트

In [18]:
# 1차 지원 에이전트
tier1_agent = Agent(
    name="Tier 1 Support",
    instructions="""
    기본적인 지원 문제를 처리합니다. 
    티켓에 노트를 추가하고 복잡한 문제는 에스컬레이션하세요.
    """
)

# 결제 전문가 에이전트
billing_specialist = Agent(
    name="Billing Specialist", 
    instructions="""
    결제, 청구, 구독 문제를 처리합니다. 
    티켓에 해결 내용을 업데이트하세요.
    """
)

# 기술 지원 에이전트
tech_specialist = Agent(
    name="Technical Support",
    instructions="""
    기술적 문제, 버그, 문제 해결을 처리합니다. 
    티켓에 해결책을 문서화하세요.
    """
)

# 트리아지 에이전트
triage_agent = Agent(
    name="Triage Agent",
    instructions="""
    지원 티켓을 분류하고 적절한 전문가에게 라우팅합니다.
    
    티켓 상태를 업데이트하고 평가 노트를 추가하세요. 
    문제 유형에 따라 라우팅:
    - 기본 문제 → Tier 1 Support
    - 결제/청구 → Billing Specialist  
    - 기술/버그 → Technical Support
    """,
    handoffs=[tier1_agent, billing_specialist, tech_specialist]
)

print("티켓 처리 에이전트가 생성되었다:")
print(f"  - {triage_agent.name}")
print(f"  - {tier1_agent.name}")
print(f"  - {billing_specialist.name}")
print(f"  - {tech_specialist.name}")

티켓 처리 에이전트가 생성되었다:
  - Triage Agent
  - Tier 1 Support
  - Billing Specialist
  - Technical Support


### 공유 컨텍스트로 티켓 처리

In [19]:
async def process_ticket_with_context(ticket: SupportTicket) -> SupportTicket:
    """
    공유 컨텍스트를 유지하면서 에이전트를 통해 티켓을 처리한다.
    
    Args:
        ticket: 처리할 지원 티켓
    
    Returns:
        처리된 지원 티켓
    """
    
    print(f"🎫 티켓 처리: {ticket.ticket_id}")
    print(f"고객: {ticket.customer_id} | 우선순위: {ticket.priority}")
    print(f"문제: {ticket.description}")
    print("-" * 60)
    
    # 단계 1: 트리아지 평가
    print("1️⃣ 트리아지 평가...")
    ticket.add_note("System", "티켓 생성 및 트리아지 할당")
    
    triage_prompt = f"""
    이 지원 티켓을 분석하세요:
    
    티켓 ID: {ticket.ticket_id}
    고객: {ticket.customer_id}
    문제 유형: {ticket.issue_type}
    우선순위: {ticket.priority}
    설명: {ticket.description}
    
    평가와 라우팅 권장 사항을 제공하세요.
    """
    
    triage_result = await Runner.run(triage_agent, triage_prompt)
    ticket.add_note("Triage Agent", f"평가: {triage_result.final_output[:100]}...")
    
    # 단계 2: 문제 유형에 따라 전문가에게 라우팅
    print("2️⃣ 전문가에게 라우팅...")
    
    issue_lower = ticket.issue_type.lower()
    desc_lower = ticket.description.lower()
    
    if "billing" in issue_lower or "payment" in desc_lower or "결제" in desc_lower or "환불" in desc_lower:
        specialist = billing_specialist
        ticket.escalate_to("Billing Specialist", "결제 관련 문제 감지")
    elif "technical" in issue_lower or "bug" in desc_lower or "크래시" in desc_lower or "오류" in desc_lower:
        specialist = tech_specialist
        ticket.escalate_to("Technical Support", "기술적 문제 감지")
    else:
        specialist = tier1_agent
        ticket.escalate_to("Tier 1 Support", "일반 지원 문제")
    
    # 단계 3: 전문가가 티켓 처리
    print(f"3️⃣ {specialist.name} 처리 중...")
    
    specialist_prompt = f"""
    전체 컨텍스트로 이 지원 티켓을 처리하세요:
    
    티켓 상세:
    - ID: {ticket.ticket_id}
    - 고객: {ticket.customer_id}  
    - 문제: {ticket.description}
    - 우선순위: {ticket.priority}
    
    이전 노트:
    {chr(10).join(ticket.resolution_notes)}
    
    에스컬레이션 기록:
    {chr(10).join(ticket.escalation_history)}
    
    해결책을 제공하고 티켓 상태를 업데이트하세요.
    """
    
    specialist_result = await Runner.run(specialist, specialist_prompt)
    ticket.add_note(specialist.name, f"해결: {specialist_result.final_output[:150]}...")
    ticket.status = "resolved"
    
    return ticket

print("process_ticket_with_context 함수가 정의되었다.")

process_ticket_with_context 함수가 정의되었다.


### 공유 컨텍스트 패턴 테스트

In [20]:
async def demo_shared_context():
    """
    여러 티켓으로 공유 컨텍스트 패턴을 시연한다.
    """
    
    # 테스트 티켓 생성
    tickets = [
        SupportTicket(
            ticket_id="TKT-001",
            customer_id="CUST-123",
            issue_type="billing",
            priority="high", 
            status="open",
            description="같은 구독에 대해 신용카드가 두 번 청구되었습니다",
            created_at=datetime.now(),
            updated_at=datetime.now()
        ),
        SupportTicket(
            ticket_id="TKT-002", 
            customer_id="CUST-456",
            issue_type="technical",
            priority="medium",
            status="open", 
            description="사진을 업로드하려고 하면 모바일 앱이 크래시됩니다",
            created_at=datetime.now(),
            updated_at=datetime.now()
        ),
        SupportTicket(
            ticket_id="TKT-003",
            customer_id="CUST-789", 
            issue_type="general",
            priority="low",
            status="open",
            description="비밀번호는 어떻게 변경하나요?",
            created_at=datetime.now(),
            updated_at=datetime.now()
        )
    ]
    
    # 각 티켓 처리
    for ticket in tickets:
        print(f"\n{'='*80}")
        processed_ticket = await process_ticket_with_context(ticket)
        
        print(f"\n📋 최종 티켓 상태:")
        print(f"상태: {processed_ticket.status}")
        print(f"에스컬레이션 레벨: {processed_ticket.escalation_level}")
        print(f"해결 노트:")
        for note in processed_ticket.resolution_notes:
            print(f"  • {note[:80]}..." if len(note) > 80 else f"  • {note}")
        print(f"에스컬레이션 기록:")
        for escalation in processed_ticket.escalation_history:
            print(f"  • {escalation}")

# 데모 실행
await demo_shared_context()


🎫 티켓 처리: TKT-001
고객: CUST-123 | 우선순위: high
문제: 같은 구독에 대해 신용카드가 두 번 청구되었습니다
------------------------------------------------------------
1️⃣ 트리아지 평가...
2️⃣ 전문가에게 라우팅...
3️⃣ Billing Specialist 처리 중...

📋 최종 티켓 상태:
상태: resolved
에스컬레이션 레벨: 1
해결 노트:
  • [System] 티켓 생성 및 트리아지 할당
  • [Triage Agent] 평가: 처리 내역 업데이트:

- 결제, 청구, 구독 문제이므로 Billing Specialist(청구/결제 전문팀)...
  • [Billing Specialist] 해결: 티켓 ID: TKT-001  
고객: CUST-123  
문제: 같은 구독에 대해 신용카드가 두 번...
에스컬레이션 기록:
  • Billing Specialist로 에스컬레이션: 결제 관련 문제 감지

🎫 티켓 처리: TKT-002
고객: CUST-456 | 우선순위: medium
문제: 사진을 업로드하려고 하면 모바일 앱이 크래시됩니다
------------------------------------------------------------
1️⃣ 트리아지 평가...
2️⃣ 전문가에게 라우팅...
3️⃣ Technical Support 처리 중...

📋 최종 티켓 상태:
상태: resolved
에스컬레이션 레벨: 1
해결 노트:
  • [System] 티켓 생성 및 트리아지 할당
  • [Triage Agent] 평가: 해결책 및 문서화 예시:

1. 원인 분석
- 문제: 사용자가 모바일 앱에서 사진을 업로드 할 때 앱이 크래시...
  • [Technical Support] 해결: 티켓 ID: TKT-002  
고객: CUST-456  
문제: 사진 업로드 시 모바일 앱 크래시  ...
에스컬레이션 기록:
  • Technical Support로 에스컬레이션: 

---

## 오케스트레이션 패턴 요약

1. 코드 기반 오케스트레이션
   - 구조화된 출력으로 워크플로우 제어
   - 명시적 제어 흐름과 에러 처리
   - 예측 가능하고 디버깅 용이
   - 사용 사례: 데이터 파이프라인, 승인 워크플로우

2. 하이브리드 오케스트레이션
   - LLM 지능 + 비즈니스 규칙 결합
   - 프로그래밍 오버라이드로 중요 결정 제어
   - AI 유연성과 비즈니스 요구사항 균형
   - 사용 사례: 고객 지원 라우팅, 에스컬레이션

3. 공유 컨텍스트 패턴
   - Pydantic 모델로 공유 상태 관리
   - 에이전트 간 상태 변경 추적
   - 협업 워크플로우 지원
   - 사용 사례: 티켓 시스템, 문서 처리

패턴 선택 가이드:
   - 예측 가능성 필요 → 코드 기반
   - 유연성 + 제어 필요 → 하이브리드
   - 상태 공유 필요 → 공유 컨텍스트

---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 오케스트레이션 패턴을 다루었다. 다음과 같은 내용을 학습하였다:

1. **코드 기반 오케스트레이션**
   - ProcessingResult 모델로 구조화된 워크플로우
   - 결정론적 에이전트 파이프라인
   - 명시적 에러 처리와 검증

2. **하이브리드 오케스트레이션**
   - LLM 기반 라우팅 결정
   - 비즈니스 규칙 오버라이드
   - 우선순위와 에스컬레이션 처리

3. **공유 컨텍스트 패턴**
   - SupportTicket 공유 비즈니스 객체
   - 에이전트 간 상태 전달
   - 해결 노트와 에스컬레이션 기록 추적

이러한 패턴을 적용하면 복잡한 멀티 에이전트 워크플로우를 효율적이고 안정적으로 구축할 수 있다.